In [ ]:
# %pip install pybaseball pandas torch tiktoken matplotlib

from pybaseball import statcast, cache as pybaseball, playerid_reverse_lookup
pybaseball.enable()

import pandas as pd
import numpy as np

pd.options.display.max_rows = 100 


In [66]:


def handy(abbrev):
    if abbrev.upper() == "R": return "right"
    if abbrev.upper() == "L": return "left"
    return "?"

def plu(howmany: int, what: str):
    return f'{howmany} {what}{"" if howmany == 1 else "s"}'

def frame(topbot):
    return "top" if topbot.lower() =="top" else "bottom"

In [65]:


games_df["inning_topbot"]

195    Top
201    Top
208    Top
215    Top
231    Top
      ... 
31     Top
33     Top
34     Top
37     Top
29     Bot
Name: inning_topbot, Length: 1000, dtype: object

In [ ]:


## the following loads all the pitch by pitch game info year by year

# for yearStr in ["2020", "2021", "2023", "2024","2022",]: 
#     print("processing: ", yearStr)
#     year_df = statcast(start_dt=f"{yearStr}-01-01", end_dt=f"{yearStr}-12-31" )
#     year_df.to_parquet(f"games/statcast-{yearStr}.parquet")


## the following goes through all pitchers and batters and gets their name

# combined_player_array = []
# for yearStr in [ "2020", "2021", "2022", "2023", "2024",]:
#     print("processing players from: ", yearStr)
#     year_df = pd.read_parquet(f"games/statcast-{yearStr}.parquet")

#     batters = year_df["batter"].unique()
#     pitchers = year_df["pitcher"].unique()

#     both_player_arrays = np.concat([batters, pitchers])

#     players_df = playerid_reverse_lookup(both_player_arrays)

#     combined_player_array.append(players_df)


# combined_player_df = pd.concat(combined_player_array).drop_duplicates()
# combined_player_df = combined_player_df.assign(
#     player_fullname = combined_player_df["name_first"].str.capitalize() + " " + combined_player_df["name_last"].str.capitalize()
# )

# all_players_df = combined_player_df[["key_mlbam", "player_fullname"]].set_index("key_mlbam")
# all_players_df.to_parquet("games/statcast-players-all.parquet",)

# all_players_df

In [ ]:

## the following prints the games dataframe scheme in n colummns

games_df = pd.read_parquet("games/statcast-2022.parquet").head(1)

rowsize = 4

for i, col in enumerate(games_df.columns.sort_values()):
    print(f"{col:<32}", end="")
    if i%rowsize == rowsize-1:
        print("") # a newline
        


In [71]:
games_df = pd.read_parquet("games/statcast-2022.parquet").head(1000) # TODO remove the head()

all_players_df = pd.read_parquet("games/statcast-players-all.parquet")
pitcher_fullname_df = all_players_df.assign(pitcher_fullname = all_players_df.player_fullname )
batter_fullname_df = all_players_df.assign(batter_fullname = all_players_df.player_fullname )

batter_named_df = pd.merge(games_df, batter_fullname_df, left_on="batter", right_on="key_mlbam")
all_pitches_df = pd.merge(batter_named_df, pitcher_fullname_df, left_on="pitcher", right_on="key_mlbam")

pitchesGroupedByGame = all_pitches_df.groupby("game_pk")

overallPitchesRecorded = 0
for game_pk, g in pitchesGroupedByGame:
    game_date, home_team, away_team = g.iloc[0][["game_date", "home_team", "away_team"]]

    print(f'On {game_date}, with {home_team} playing {away_team} at home in gamae {game_pk}')
    
    for (inning, inning_topbot), i in g.groupby(["inning", "inning_topbot"], sort=True):
        print(f'In the {frame(inning_topbot)} of inning {inning}')

        for (at_bat_number), ab in i.groupby("at_bat_number", sort=True):

            # get atbat info from the first pitch, fp
            fp = ab.iloc[0]

            on_base, baserunners = [], ""
            if pd.notna(fp.on_1b): on_base.append(f"{fp.on_1b} on first")
            if pd.notna(fp.on_2b): on_base.append(f"{fp.on_2b} on second")
            if pd.notna(fp.on_3b): on_base.append(f"{fp.on_3b} on third")
            if len(on_base)>0:
                baserunners = f"with {', '.join(on_base)}"

            print(
                f'{handy(fp.p_throws)} handed pitcher {fp.pitcher_fullname}',
                f'faces {handy(fp.stand)} handed batter {fp.batter_fullname},',
                f'{home_team} {fp.home_score}, {away_team} {fp.away_score} {baserunners}',
            )

            for p in ab.sort_values(["pitch_number"]).itertuples():
                # TODO consider dealing with plurals for the count and outs
                print(
                    f'pitch from {p.pitcher_fullname} with {plu(p.balls, "ball")}',
                    f'{plu(p.strikes, "strike")} and {plu(p.outs_when_up, "out")}',
                    f'to {p.batter_fullname}',                    
                    f'is a {p.pitch_name} {p.description}'
                , end="")

                if p.post_bat_score > p.bat_score:
                    print(f" {p.post_bat_score - p.bat_score} runs scored")
                else:
                    print() # just terminate the p status




On 2022-11-05 00:00:00, with HOU playing PHI at home in gamae 715719
In the bottom of inning 1
right handed pitcher Zack Wheeler faces right handed batter José Altuve, HOU 0, PHI 0 
pitch from Zack Wheeler with 0 balls 0 strikes and 0 outs to José Altuve is a 4-Seam Fastball ball
pitch from Zack Wheeler with 1 ball 0 strikes and 0 outs to José Altuve is a 4-Seam Fastball called_strike
pitch from Zack Wheeler with 1 ball 1 strike and 0 outs to José Altuve is a Curveball called_strike
pitch from Zack Wheeler with 1 ball 2 strikes and 0 outs to José Altuve is a 4-Seam Fastball swinging_strike
right handed pitcher Zack Wheeler faces right handed batter Jeremy Peña, HOU 0, PHI 0 
pitch from Zack Wheeler with 0 balls 0 strikes and 1 out to Jeremy Peña is a 4-Seam Fastball ball
pitch from Zack Wheeler with 1 ball 0 strikes and 1 out to Jeremy Peña is a Sinker hit_into_play
right handed pitcher Zack Wheeler faces left handed batter Yordan Álvarez, HOU 0, PHI 0 
pitch from Zack Wheeler with 0 b